In [1]:
import pandas as pd
import glob
from tqdm import tqdm
tqdm.pandas()

import stanza
stanza.download('hu')
nlp = stanza.Pipeline('hu')

import networkx as nx
import numpy as np

2020-07-19 18:31:28 INFO: Downloading default packages for language: hu (Hungarian)...
2020-07-19 18:31:29 INFO: File exists: /mnt/volume/jupyter/stanza_resources/hu/default.zip.
2020-07-19 18:31:33 INFO: Finished downloading models and saved to /mnt/volume/jupyter/stanza_resources.
2020-07-19 18:31:33 INFO: Loading these models for language: hu (Hungarian):
| Processor | Package |
-----------------------
| tokenize  | szeged  |
| pos       | szeged  |
| lemma     | szeged  |
| depparse  | szeged  |

2020-07-19 18:31:33 INFO: Use device: cpu
2020-07-19 18:31:33 INFO: Loading: tokenize
2020-07-19 18:31:33 INFO: Loading: pos
2020-07-19 18:31:35 INFO: Loading: lemma
2020-07-19 18:31:35 INFO: Loading: depparse
2020-07-19 18:31:36 INFO: Done loading processors!


In [2]:
def lemmatize(text):
    if type(text) is float: return text
    doc=nlp(text)
    return ' '.join([word.lemma for word in doc.sentences[0].words])

In [3]:
limit=20

In [4]:
resfiles=glob.glob('resultfiles/*_szokereso_result.csv')
dictstrings = ['person_data', 'wikilist','settlement_list','stanza','szotar_2.1']

In [5]:
res={}
for resfile in resfiles:
    df=pd.read_csv(resfile)
    freqdf = pd.concat([df[col]
                       for dictstring in dictstrings
                       for col in df.columns if dictstring in col]
                       ).value_counts(
                       ).rename_axis('unique_entries_not_lemmatized'
                       ).reset_index(name='counts')
    res[resfile]=freqdf

In [6]:
freqdf=res['resultfiles/data_2020-07-06_06:00:32_szokereso_result.csv']

In [7]:
freqdf['lemmatized']=freqdf.truncate(after=limit)['unique_entries_not_lemmatized'].progress_apply(lemmatize)

100%|██████████| 21/21 [00:41<00:00,  1.98s/it]


In [8]:
freqdf_onlylemmatized = freqdf.\
                        groupby(['lemmatized']).\
                        agg({'counts': 'sum'}).\
                        sort_values(by=['counts'],ascending=False).\
                        reset_index()

In [9]:
cooc = pd.DataFrame(index=freqdf_onlylemmatized['lemmatized'], columns=freqdf_onlylemmatized['lemmatized'])

In [10]:
cooc.index.names =   ['row']
cooc.columns.names = ['column']

In [11]:
df=pd.read_csv('resultfiles/data_2020-07-06_06:00:32_szokereso_result.csv')

In [12]:
dictcols = [col for col in df.columns if any([dictstring in col for dictstring in dictstrings])]

In [13]:
for icoocrow, coocrow in enumerate(cooc.index):
    for icooccol, cooccol in enumerate(cooc.columns):
        if icoocrow ==icooccol: print(icoocrow/limit)
        if icoocrow > icooccol:
            count=0
            for index, row in df[[*dictcols]].iterrows():
                if cooccol in list(row) and coocrow in list(row):
                    count+=1
            cooc.loc[coocrow,cooccol]=count

0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95


In [14]:
#!!!RUN ONLY ONCE!!!
cooc=cooc.add(cooc.transpose(),fill_value=0)

In [53]:
f = open("toR2.txt", "w")
for i, col in enumerate(cooc.columns):
    for j, index in enumerate(cooc.index):
        if i!=j:
            sig=cooc.loc[col,index]
            if sig != 0:
                print(col, index, sig)
                f.write('|'.join([str(each) for each in [col, index, sig, '\n']]))
f.close()

Orbán Viktor Varga Mihály 5
Orbán Viktor Szijjártó Péter 7
Orbán Viktor Karácsony Gergely 16
Orbán Viktor Egyesült Államok 12
Orbán Viktor Donald Trump 19
Orbán Viktor Gulyás Gergely 2
Orbán Viktor Hollik István 4
Orbán Viktor Novák Katalin 2
Orbán Viktor Bencsik János 2
Varga Mihály Orbán Viktor 5
Varga Mihály Szijjártó Péter 1
Varga Mihály Karácsony Gergely 8
Varga Mihály Tállai András 3
Varga Mihály Gulyás Gergely 3
Varga Mihály Hollik István 2
Szijjártó Péter Orbán Viktor 7
Szijjártó Péter Varga Mihály 1
Szijjártó Péter Karácsony Gergely 2
Szijjártó Péter Egyesült Államok 1
Szijjártó Péter Novák Katalin 1
Karácsony Gergely Orbán Viktor 16
Karácsony Gergely Varga Mihály 8
Karácsony Gergely Szijjártó Péter 2
Karácsony Gergely Gulyás Gergely 4
Karácsony Gergely Hollik István 16
Karácsony Gergely Novák Katalin 1
Karácsony Gergely Bencsik János 1
Egyesült Államok Orbán Viktor 12
Egyesült Államok Szijjártó Péter 1
Egyesült Államok ##### EXTRA 1
Egyesült Államok Donald Trump 21
Egyesült Á